In [1]:
from pathlib import Path
import os
from collections import Counter
import numpy as np
import time
from tqdm.notebook import tqdm

np.random.seed(8080)

data_path = Path("/content/drive/My Drive/Adv Projects in ML/data/bpe")
print(data_path)
print(os.listdir(data_path))

os.chdir("/content/drive/My Drive/Adv Projects in ML/")

!nvidia-smi

/content/drive/My Drive/Adv Projects in ML/data/bpe
['split_val_bpe.lang1', 'split_val_bpe.lang2', 'codes.file', 'split_train_bpe.lang1', 'split_train_bpe_d.lang1', 'split_train_bpe_d.lang2', 'split_train_bpe.lang2']
Wed Apr  8 20:07:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                               

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from transformer import Transformer, CustomSchedule, create_masks

print("Tensorflow version " + tf.__version__)

tf.random.set_seed(8080)
# make sure numpy seeded

Tensorflow version 2.2.0-rc2


In [3]:
# read data
with open(data_path/"split_train_bpe.lang1","r") as f:
    english = f.read()
print(len(english.split("\n")), english[:200])
    
with open(data_path/"split_train_bpe.lang2","r") as f:
    french = f.read()
print(len(french.split("\n")), french[:200])

with open(data_path/"split_val_bpe.lang1","r") as f:
    english_val = f.read()
print(len(english_val.split("\n")), english_val[:200])

with open(data_path/"split_val_bpe.lang2","r") as f:
    french_val = f.read()
print(len(french_val.split("\n")), french_val[:200])

# create vocab
english_vocab = list(set(english.replace("\n", " <eos> ").split()))
french_vocab = list(set(french.replace("\n", " <eos> ").split()))
len(english_vocab), len(french_vocab)

english_counter = Counter(english.replace("\n", " <eos> ").split())
french_counter = Counter(french.replace("\n", " <eos> ").split())
len(english_counter), len(french_counter)

english_counter.update({"<unk>":0})
french_counter.update({"<unk>":0})
english_counter.update({"<start>":0})
french_counter.update({"<start>":0})
len(english_counter), len(french_counter)

english_vocab = list(english_counter.keys())
french_vocab = list(french_counter.keys())

# # trim vocab to 10k+2, 12k+2
# english_vocab = ["<start>","<unk>"]
# for i in english_counter.most_common(10000):
#   english_vocab.append(i[0])
# french_vocab = ["<start>","<unk>"]
# for i in french_counter.most_common(12000):
#   french_vocab.append(i[0])

english_word2id = {}
english_id2word = {}
french_word2id = {}
french_id2word = {}

# start enumerate from 1 so that 0 is reserved for padding seqs 
for i, w in enumerate(english_vocab, start=1):
  english_word2id[w] = i
  english_id2word[i] = w

for i, w in enumerate(french_vocab, start=1):
  french_word2id[w] = i
  french_id2word[i] = w

len(english_word2id), len(english_id2word), len(french_word2id), len(french_id2word)

def transform_data(english_lang1, french_lang2):
  english_lines = english_lang1.split("\n")
  french_lines = french_lang2.split("\n")

  data_english = []
  data_french = []

  for line in english_lines:
    line2id = [english_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(english_word2id[word])
      except:
        line2id.append(english_word2id["<unk>"])
    line2id.append(english_word2id["<eos>"])
    data_english.append(line2id)

  for line in french_lines:
    line2id = [french_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(french_word2id[word])
      except:
        line2id.append(french_word2id["<unk>"])
    line2id.append(french_word2id["<eos>"])
    data_french.append(line2id)

  print(len(data_english), len(data_french))
  return data_english, data_french

data_english, data_french = transform_data(english, french)
data_english_val, data_french_val = transform_data(english_val, french_val)

english_id2word[54], len(data_english), len(data_french), len(data_english_val), len(data_french_val), len(english_vocab), len(french_vocab)

8800 as mr de castro is not present mr le fol@@ l who is re@@ placing mr de castro has the floor
on the other hand if you 're visiting an under@@ developed country and 25 dollars bu@@ ys you a gour@@ met m
8800 Comme M. De Castro est absent , M. Le F@@ ol@@ l , qui le remplace , a la parole .
D' un autre côté , si vous êtes dans un pays en voie de développement , où 25 dollars peuvent vous obtenir un repas d
2200 what action does the council intend to take in the face of this seriously discriminatory attitude which runs contrary to the principles of the eu
where would you like to go next
if that were not enoug
2200 Quelles mesures le Conseil comp@@ te-@@ t-il adopter face à cette attitude qui constitue une grave discrimination et est contraire aux principes sur lesquels l' Union européenne est fondée ?
Où souhai
8800 8800
2200 2200


('consideration', 8800, 8800, 2200, 2200, 10927, 13658)

In [0]:
np.savez("data_and_vocab_bpe.npz", data_english=data_english, data_french=data_french, data_english_val=data_english_val, data_french_val=data_french_val, 
         english_word2id=english_word2id, english_id2word=english_id2word, french_word2id=french_word2id,french_id2word=french_id2word)

In [16]:
BUFFER_SIZE = len(data_english)
BATCH_SIZE = 64
EPOCHS = 50
print("No. of training batches: ", np.ceil(len(data_english)/BATCH_SIZE))
print("No. of validation batches: ", np.ceil(len(data_english_val)/BATCH_SIZE))

# transformer hyperparams
num_layers = 4
d_model = 256
dff = 512
num_heads = 8
input_vocab_size = len(english_vocab) + 1
target_vocab_size = len(french_vocab) + 1
dropout_rate = 0.3
pe_input = max(max([len(i) for i in data_english]),max([len(i) for i in data_english_val]))
pe_target = max(max([len(i) for i in data_french]),max([len(i) for i in data_french_val]))

No. of training batches:  138.0
No. of validation batches:  35.0


In [0]:
tensor_train = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french, padding='post')
)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)
tensor_val = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english_val, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french_val, padding='post')
)).batch(BATCH_SIZE, drop_remainder=False)

In [19]:
transformer = Transformer(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, 
    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, 
    pe_input=pe_input, pe_target=pe_target, rate=dropout_rate)

temp_input = tf.random.uniform((BATCH_SIZE, pe_input), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((BATCH_SIZE, pe_target), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([64, 130, 13659])

In [20]:
transformer.summary()

Model: "transformer_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (Encoder)          multiple                  4905984   
_________________________________________________________________
decoder_1 (Decoder)          multiple                  6659840   
_________________________________________________________________
dense_129 (Dense)            multiple                  3510363   
Total params: 15,076,187
Trainable params: 15,076,187
Non-trainable params: 0
_________________________________________________________________


In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_loss = tf.keras.metrics.Mean(name='loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='val_accuracy')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [0]:
experiment_number = "bpe_2"

checkpoint_path = "./checkpoints/train"+experiment_number

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

writer_train = tf.summary.create_file_writer("log_dir/"+experiment_number)

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

@tf.function(input_signature=train_step_signature)
def val_step(inp, tar):
  print(inp.get_shape(), tar.get_shape())
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  predictions, _ = transformer(inp, tar_inp, 
                                False, 
                                enc_padding_mask, 
                                combined_mask, 
                                dec_padding_mask)
  loss = loss_function(tar_real, predictions)
  
  val_loss(loss)
  val_accuracy(tar_real, predictions)


In [24]:
best_val_loss = np.inf

for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  val_loss.reset_states()
  val_accuracy.reset_states()
  
  for (batch, (inp, tar)) in tqdm(enumerate(tensor_train)):
    train_step(inp, tar)
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Training Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    
  print ('Epoch {} Training Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

  with writer_train.as_default():
    tf.summary.scalar('train_loss', train_loss.result(), step=epoch)

  print("validating")
  for (batch, (inp, tar)) in enumerate(tensor_val):
    val_step(inp, tar)
  
  print ('Epoch {} Validation Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                            val_loss.result(), 
                                            val_accuracy.result()))
  if best_val_loss > val_loss.result():
    best_val_loss = val_loss.result()
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
  with writer_train.as_default():
    tf.summary.scalar('val_loss', val_loss.result(), step=epoch)

Epoch 1 Batch 0 Training Loss 9.5188 Accuracy 0.0000
Epoch 1 Batch 50 Training Loss 9.4604 Accuracy 0.0012
Epoch 1 Batch 100 Training Loss 9.3506 Accuracy 0.0045

Epoch 1 Training Loss 9.2534 Accuracy 0.0054
Time taken for 1 epoch: 44.70581364631653 secs

validating
(None, None) (None, None)
Epoch 1 Validation Loss 8.7876 Accuracy 0.0076
Saving checkpoint for epoch 1 at ./checkpoints/trainbpe_2/ckpt-1


Epoch 2 Batch 0 Training Loss 8.8391 Accuracy 0.0070
Epoch 2 Batch 50 Training Loss 8.6264 Accuracy 0.0075
Epoch 2 Batch 100 Training Loss 8.3885 Accuracy 0.0075

Epoch 2 Training Loss 8.2025 Accuracy 0.0075
Time taken for 1 epoch: 34.225117683410645 secs

validating
Epoch 2 Validation Loss 7.4580 Accuracy 0.0076
Saving checkpoint for epoch 2 at ./checkpoints/trainbpe_2/ckpt-2


Epoch 3 Batch 0 Training Loss 7.4792 Accuracy 0.0044
Epoch 3 Batch 50 Training Loss 7.2553 Accuracy 0.0076
Epoch 3 Batch 100 Training Loss 7.0925 Accuracy 0.0092

Epoch 3 Training Loss 7.0157 Accuracy 0.0114
Time taken for 1 epoch: 33.97651672363281 secs

validating
Epoch 3 Validation Loss 6.8168 Accuracy 0.0215
Saving checkpoint for epoch 3 at ./checkpoints/trainbpe_2/ckpt-3


Epoch 4 Batch 0 Training Loss 6.6251 Accuracy 0.0179
Epoch 4 Batch 50 Training Loss 6.6956 Accuracy 0.0202
Epoch 4 Batch 100 Training Loss 6.6593 Accuracy 0.0202

Epoch 4 Training Loss 6.6312 Accuracy 0.0203
Time taken for 1 epoch: 33.924625873565674 secs

validating
Epoch 4 Validation Loss 6.5866 Accuracy 0.0227
Saving checkpoint for epoch 4 at ./checkpoints/trainbpe_2/ckpt-4


Epoch 5 Batch 0 Training Loss 6.4798 Accuracy 0.0199
Epoch 5 Batch 50 Training Loss 6.4208 Accuracy 0.0219
Epoch 5 Batch 100 Training Loss 6.3721 Accuracy 0.0228

Epoch 5 Training Loss 6.3330 Accuracy 0.0232
Time taken for 1 epoch: 33.805588245391846 secs

validating
Epoch 5 Validation Loss 6.2512 Accuracy 0.0269
Saving checkpoint for epoch 5 at ./checkpoints/trainbpe_2/ckpt-5


...

Epoch 28 Batch 0 Training Loss 2.8273 Accuracy 0.0757
Epoch 28 Batch 50 Training Loss 2.7364 Accuracy 0.0773
Epoch 28 Batch 100 Training Loss 2.7985 Accuracy 0.0750

Epoch 28 Training Loss 2.8371 Accuracy 0.0739
Time taken for 1 epoch: 33.29198622703552 secs

validating
Epoch 28 Validation Loss 5.3188 Accuracy 0.0483


Epoch 29 Batch 0 Training Loss 2.6254 Accuracy 0.0854
Epoch 29 Batch 50 Training Loss 2.6060 Accuracy 0.0800
Epoch 29 Batch 100 Training Loss 2.6645 Accuracy 0.0786

Epoch 29 Training Loss 2.7131 Accuracy 0.0775
Time taken for 1 epoch: 33.28731417655945 secs

validating
Epoch 29 Validation Loss 5.3720 Accuracy 0.0481


Epoch 30 Batch 0 Training Loss 2.3880 Accuracy 0.0876
Epoch 30 Batch 50 Training Loss 2.4703 Accuracy 0.0849
Epoch 30 Batch 100 Training Loss 2.5391 Accuracy 0.0825

Epoch 30 Training Loss 2.5841 Accuracy 0.0809
Time taken for 1 epoch: 33.28516411781311 secs

validating
Epoch 30 Validation Loss 5.4708 Accuracy 0.0476


Epoch 31 Batch 0 Training Loss 2.2382 Accuracy 0.0910
Epoch 31 Batch 50 Training Loss 2.3263 Accuracy 0.0892
Epoch 31 Batch 100 Training Loss 2.3953 Accuracy 0.0871

Epoch 31 Training Loss 2.4429 Accuracy 0.0854
Time taken for 1 epoch: 33.23005557060242 secs

validating


KeyboardInterrupt: ignored

Evaluate best model

In [25]:
# load model

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [26]:
transformer

In [27]:
val_loss.reset_states()
val_accuracy.reset_states()
  
for (batch, (inp, tar)) in enumerate(tensor_val):
  val_step(inp, tar)
  
print ('Validation Loss {:.4f} Accuracy {:.4f}'.format(
                                          val_loss.result(), 
                                          val_accuracy.result()))
  

Validation Loss 4.9450 Accuracy 0.0482


In [0]:
def generate_predictions(inp_sentences):

  if len(inp_sentences.get_shape())==1:
    encoder_input = tf.expand_dims(inp_sentences, 0)
    decoder_input = [french_word2id["<start>"]]
    output = tf.expand_dims(decoder_input, 0)

  else:
    encoder_input = inp_sentences
    decoder_input = [french_word2id["<start>"]]*inp_sentences.get_shape()[0]
    output = tf.expand_dims(decoder_input, -1)


  # encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # decoder_input = [french_word2id["<start>"]]
  # output = tf.expand_dims(decoder_input, 0)
    
  for i in range(pe_target):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # # return the result if the predicted_id is equal to the end token
    # if predicted_id == french_word2id["<eos>"]:
    #   return tf.squeeze(output, axis=0), attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  # return tf.squeeze(output, axis=0), attention_weights
  return output, attention_weights

In [29]:
all_preds = []
for (batch_i, (inp, tar)) in tqdm(enumerate(tensor_val)):

  preds, attention = generate_predictions(inp)
  all_preds.append(preds)

In [30]:
translated_sentences = []

for k in tqdm(all_preds):
  for i in k:
    sentence_french = []
    for j in i.numpy()[1:]:
      if j==0 or j==french_word2id["<eos>"]:
        break
      sentence_french.append(french_id2word[j])

    sentence_french = " ".join(sentence_french)

    translated_sentences.append(sentence_french)

translated_sentences = "\n".join(translated_sentences)

with open("predictions.txt","w") as f:
  f.write(translated_sentences)